# NOAA Precipitation Data
### Leena Chen

This is a brief excerpt of a R project completed for Applied Statistical Computing in spring 2020. In the project, I explored annual and decade precipitation data over time from three weather stations in Alabama. In this excerpt, I clean, format, and merge the data into a rectangular dataframe, which can be useful in a wide variety of applications. 

I've included a lot of markdown comments narrating my process to make this as accessible as possible. Feel free to read it if it's helpful, or feel free to just skip over it instead!

The weather data used here was collected by NOAA: https://www.ncdc.noaa.gov/ushcn/introduction

### Reading in and exploring the data

In [1]:
metadata <- read.csv("C:/Users/eilee/Downloads/hw5_spatial/station_metadata.csv")
brewton_prcp <- read.csv("C:/Users/eilee/Downloads/hw5_spatial/USH00011084.prcp.csv")
fairhope_prcp <- read.csv("C:/Users/eilee/Downloads/hw5_spatial/USH00012813.prcp.csv")
gainesville_prcp <- read.csv("C:/Users/eilee/Downloads/hw5_spatial/USH00013160.prcp.csv")

In [2]:
colnames(metadata)

[1] "component_1"     "component_2"     "component_3"     "coop_id"        
 [5] "country_code"    "elevation"       "id"              "id_place_holder"
 [9] "latitude"        "longitude"       "name"            "network_code"   
[13] "state"           "utc_offset"

In [3]:
head(metadata)
head(brewton_prcp)
head(fairhope_prcp)
head(gainesville_prcp)

component_1,component_2,component_3,coop_id,country_code,elevation,id,id_place_holder,latitude,longitude,name,network_code,state,utc_offset
------,------,------,11084,US,25.9,USH00011084,0,31.0581,-87.0547,BREWTON 3 SSE,H,AL,6
------,------,------,12813,US,7.0,USH00012813,0,30.5467,-87.8808,FAIRHOPE 2 NE,H,AL,6
011694,------,------,13160,US,38.1,USH00013160,0,32.8347,-88.1342,GAINESVILLE LOCK,H,AL,6
------,------,------,13511,US,67.1,USH00013511,0,32.7017,-87.5808,GREENSBORO,H,AL,6
------,------,------,13816,US,132.0,USH00013816,0,31.8700,-86.2542,HIGHLAND HOME,H,AL,6
------,------,------,15749,US,164.6,USH00015749,0,34.7442,-87.5997,MUSCLE SHOALS AP,H,AL,6


X,dmflag0,dmflag1,dmflag10,dmflag11,dmflag2,dmflag3,dmflag4,dmflag5,dmflag6,...,value11,value2,value3,value4,value5,value6,value7,value8,value9,year
0,,,,,,,,,,...,432,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,1890
1,,,,,,,,,,...,1333,1461,419,699,1702,1080,437,508,0,1891
2,,,,,,,,,,...,-9999,650,406,965,1981,3114,3442,762,254,1892
3,,,,,,,,,,...,1237,1181,965,1854,876,1080,1638,876,1613,1893
4,,,,,,,,,,...,191,2002,572,1181,673,-9999,-9999,-9999,76,1894
5,,,,,,,,,,...,762,1016,1016,1016,1016,762,-9999,-9999,508,1895


X,dmflag0,dmflag1,dmflag10,dmflag11,dmflag2,dmflag3,dmflag4,dmflag5,dmflag6,...,value11,value2,value3,value4,value5,value6,value7,value8,value9,year
0,,,,,,,,,,...,-9999,-9999,-9999,-9999,-9999,-9999,304,-9999,-9999,1917
1,,,,,,,,,,...,1996,226,2824,1045,938,823,-9999,1072,2690,1918
2,,,,,,,,,,...,888,1537,1326,1706,470,3415,2398,710,2936,1919
3,,,,,,,,a,,...,1267,560,1825,908,1967,2291,2172,3526,401,1920
4,,,,,,,,,,...,1024,1478,1234,958,186,2266,2022,686,435,1921
5,,,,g,,,,,,...,2535,1765,396,2616,1415,1454,2002,1052,754,1922


X,dmflag0,dmflag1,dmflag10,dmflag11,dmflag2,dmflag3,dmflag4,dmflag5,dmflag6,...,value11,value2,value3,value4,value5,value6,value7,value8,value9,year
0,.,,.,.,b,,.,.,.,...,582,591,561,1760,935,1572,51,676,307,1893
1,.,.,.,.,.,,,,,...,1201,1240,1222,549,673,480,2080,216,752,1894
2,.,.,,,.,,,,,...,981,2322,498,826,574,1293,668,254,533,1895
3,,,,,,,,,,...,589,1553,1249,1490,1694,1018,407,157,446,1896
4,,,,,,,,,,...,266,1921,352,102,94,142,267,178,13,1897
5,,,.,.,,,,,,...,937,547,757,25,521,1025,822,167,529,1898


Notice that "------" represents missing values in the metadata file, and "-9999" represents missing values in the files that contain data for each station.

In [4]:
dim(metadata)

[1] 1218   14

There are 1218 stations total according to the metadata, but we'll only look at 3 of these stations for the purposes of this project: Brewton, Fairhope, and Gainesville in Alabama.

In [5]:
colnames(brewton_prcp)

[1] "X"        "dmflag0"  "dmflag1"  "dmflag10" "dmflag11" "dmflag2" 
 [7] "dmflag3"  "dmflag4"  "dmflag5"  "dmflag6"  "dmflag7"  "dmflag8" 
[13] "dmflag9"  "dsflag0"  "dsflag1"  "dsflag10" "dsflag11" "dsflag2" 
[19] "dsflag3"  "dsflag4"  "dsflag5"  "dsflag6"  "dsflag7"  "dsflag8" 
[25] "dsflag9"  "id"       "qcflag0"  "qcflag1"  "qcflag10" "qcflag11"
[31] "qcflag2"  "qcflag3"  "qcflag4"  "qcflag5"  "qcflag6"  "qcflag7" 
[37] "qcflag8"  "qcflag9"  "value0"   "value1"   "value10"  "value11" 
[43] "value2"   "value3"   "value4"   "value5"   "value6"   "value7"  
[49] "value8"   "value9"   "year"

### Cleaning the data and creating a dataframe of annual total precipitation

Since we are only working with three stations, I chose to manually type out the following three similar processes to generate a dataframe of annual total precipitation for each station. The three processes are nearly identical, with the only changes being in the names of the stations, so they could easily be generalized into a function with a little text manipulation work. It didn't seem worth the time writing the function for only three stations; however, if I were to calculate annual total precipitation for each of the 1218 total weather stations, it would definitely be more efficient to write a function. 

In [6]:
brewton_prcp_cols <- c('value0', 'value1', 'value10', 'value11', 'value2', 'value3', 
                     'value4', 'value5', 'value6', 'value7', 'value8', 'value9')
brewton_prcp_missing <- brewton_prcp[, brewton_prcp_cols]

brewton_missing_values <- brewton_prcp_missing < 0
brewton_prcp_no_missing <- replace(brewton_prcp_missing, brewton_missing_values, NA)

brewton_prcp_totals <- apply(brewton_prcp_no_missing, 1 , sum, na.rm = TRUE)

brewton_totals_year <- cbind(year = brewton_prcp$year, brewton_precip = brewton_prcp_totals)

In [7]:
fairhope_prcp_cols <- c('value0', 'value1', 'value10', 'value11', 'value2', 'value3', 
                     'value4', 'value5', 'value6', 'value7', 'value8', 'value9')
fairhope_prcp_missing <- fairhope_prcp[, fairhope_prcp_cols]

fairhope_missing_values <- fairhope_prcp_missing < 0
fairhope_prcp_no_missing <- replace(fairhope_prcp_missing, fairhope_missing_values, NA)

fairhope_prcp_totals <- apply(fairhope_prcp_no_missing, 1 , sum, na.rm = TRUE)

fairhope_totals_year <- cbind(year = fairhope_prcp$year, fairhope_precip = fairhope_prcp_totals)

In [8]:
gainesville_prcp_cols <- c('value0', 'value1', 'value10', 'value11', 'value2', 'value3', 
                     'value4', 'value5', 'value6', 'value7', 'value8', 'value9')
gainesville_prcp_missing <- gainesville_prcp[, gainesville_prcp_cols]

gainesville_missing_values <- gainesville_prcp_missing < 0
gainesville_prcp_no_missing <- replace(gainesville_prcp_missing, gainesville_missing_values, NA)

gainesville_prcp_totals <- apply(gainesville_prcp_no_missing, 1 , sum, na.rm = TRUE)

gainesville_totals_year <- cbind(year = gainesville_prcp$year, gainesville_precip = gainesville_prcp_totals)

Now, let's merge each of the three separate dataframes into one that contains annual precipitation for all three stations. 

It's important to make sure to merge by year to match the rows up according to the year the correspond to because the different stations have different annual data. For example, based on our data exploration above, we know that the Brewton station has precipitation data from as early as the year 1890, whereas the other two stations started recording data later.  

In [9]:
brewton_and_fairhope_annual_precip <- merge(brewton_totals_year, fairhope_totals_year, by = "year", all = TRUE)
all_stations_annual_precip <- merge(brewton_and_fairhope_annual_precip, gainesville_totals_year, by = "year", all = TRUE)

Let's take a peek at what our final combined dataset looks like! 

In [10]:
head(all_stations_annual_precip)
tail(all_stations_annual_precip)

year,brewton_precip,fairhope_precip,gainesville_precip
1890,432,NA,NA
1891,12823,NA,NA
1892,16273,NA,NA
1893,15460,NA,9928
1894,9493,NA,11026
1895,6477,NA,11021


,year,brewton_precip,fairhope_precip,gainesville_precip
125,2014,16408,21398,14235
126,2015,NA,17883,12332
127,2016,NA,15896,10054
128,2017,NA,21074,16161
129,2018,NA,21323,17230
130,2019,NA,9887,11660


### Exploring the combined dataframe of total annual precipitation

In [11]:
dim(all_stations_annual_precip)

[1] 130   4

In [12]:
max(all_stations_annual_precip$year) - min(all_stations_annual_precip$year) 

[1] 129

In [13]:
sum(is.na(all_stations_annual_precip))

[1] 62

Looks like the dimensions of the final combined dataframe are 130 rows by 4 columns. This sounds about right, since we have one column dedicated to years and the other three columns dedicated to data from each of the three stations. 

Notice that there's a discrepancy between the number of rows and the range of years in the dataframe. This may appear concerning at first, but it's not an error! Recall that substraction gives us the span of the values, which is total value minus one. 

So, it looks like we're good on the dimensions of the final dataframe! At least based on this inspection, no data was lost in the merging process. 

However, note that there are 62 missing values total, both from the data cleaning step and from the merging step (not every station collected precipitation data for every year). 

### Creating a dataframe of decade total precipitation

Building on our dataframe of annual total precipitation, we can also calculate decade total precipitation as well. 

In [14]:
all_decade <- cut(all_stations_annual_precip$year, breaks = seq(1880, 2020, by = 10), include.lowest = FALSE, dig.lab = 10)

In [15]:
brewton_precipitation <- tapply(all_stations_annual_precip$brewton_precip, all_decade, sum, na.rm = TRUE)
fairhope_precipitation <- tapply(all_stations_annual_precip$fairhope_precip, all_decade, sum, na.rm = TRUE)
gainesville_precipitation <- tapply(all_stations_annual_precip$gainesville_precip, all_decade, sum, na.rm = TRUE)

In [16]:
all_stations_decade_precip_missing <- cbind(brewton_precipitation, fairhope_precipitation, gainesville_precipitation)

missing_decade <- all_stations_decade_precip_missing < 1

all_stations_decade_precip <- replace(all_stations_decade_precip_missing, missing_decade, NA)

Here's what the final dataframe looks like! This time, instead of just printing the beginning and the end, we can safely take a look at the entire dataframe of only 14 rows of data. 

In [17]:
all_stations_decade_precip

,brewton_precipitation,fairhope_precipitation,gainesville_precipitation
"(1880,1890]",432,NA,NA
"(1890,1900]",102385,NA,83883
"(1900,1910]",NA,NA,134087
"(1910,1920]",NA,53333,141051
"(1920,1930]",83086,157771,134748
"(1930,1940]",147280,151661,142845
"(1940,1950]",170332,184846,149687
"(1950,1960]",142295,158746,128033
"(1960,1970]",160448,156198,142495
"(1970,1980]",169215,172170,165982


### So we've cleaned, formatted, and merged the data. What now?

Well, now that we have the data we need, there are plenty of places to go from here. It all comes down to what we're interested in finding. We could visualize the precipitation data from each of the three stations over time, we could take a deeper dive into individual years or decades, we could treat the data as representative of Alabama weather and compare it against that of other states, and so on and so forth! 